In [1]:
from parameters.dataset import BPI2012Parameters
from dataset import BPI2012Dataset
from utils.print import print_block
from utils.bpi2012 import remove_trail_steps, print_model_prediction_result, remove_tags_for_seq
import tensorflow as tf
from model import LSTMPredNextModel, LSTMScenarioCfModel
import numpy as np
from dice import EventLogDiCE
from parameters.training import TrainingParameters, LossParameters, OptimizerParameters
from utils.preprocessing import dataset_split

In [2]:
dataset = BPI2012Dataset(BPI2012Parameters()) 


| Preprocessed data loaded successfully: ./data/preprocessed/BPI_Challenge_2012_with_resource/AOW_CompleteOnly 


In [3]:
### Prepare the training dataset.
train_params = TrainingParameters(
    stop_epoch=20,
)
train_dataset, test_dataset, validation_dataset = dataset_split(
    list(range(len(dataset))),
    train_params.train_test_split_portion,
    seed=train_params.random_seed,
    shuffle=True
)
training_df = dataset.df.iloc[train_dataset]

In [4]:
#### Get the trace
trace_idx = 52
_, example_activities, _, example_resources, example_amount, _ = dataset.collate_fn([trace_idx])
print_block(dataset.activity_vocab.list_of_index_to_vocab_2d(example_activities),"Activity")
print_block(dataset.resource_vocab.list_of_index_to_vocab_2d(example_resources),"Resource")


====================Activity====================
| [['<SOS>', 'A_SUBMITTED_COMPLETE', 'A_PARTLYSUBMITTED_COMPLETE', 'A_PREACCEPTED_COMPLETE', 'W_Completeren aanvraag_COMPLETE', 'W_Completeren aanvraag_COMPLETE', 'A_ACCEPTED_COMPLETE', 'A_FINALIZED_COMPLETE', 'O_SELECTED_COMPLETE', 'O_CREATED_COMPLETE', 'O_SENT_COMPLETE', 'W_Completeren aanvraag_COMPLETE', 'O_SENT_BACK_COMPLETE', 'W_Nabellen offertes_COMPLETE', 'O_ACCEPTED_COMPLETE', 'A_APPROVED_COMPLETE', 'A_REGISTERED_COMPLETE', 'A_ACTIVATED_COMPLETE', 'W_Valideren aanvraag_COMPLETE']] 

====================Resource====================
| [['<SOS>', '112', '112', '112', '11180', '11201', '11201', '11201', '11201', '11201', '11201', '11201', '11049', '11049', '10629', '10629', '10629', '10629', '10629']] 


In [5]:
## Remove tails
tail_length_to_remove = 13
example_idx_activities, example_idx_resources = remove_trail_steps(example_activities, example_resources, tail_length_to_remove)
example_vocab_activities = dataset.activity_vocab.list_of_index_to_vocab_2d(example_idx_activities)[0]
example_vocab_resources = dataset.resource_vocab.list_of_index_to_vocab_2d(example_idx_resources)[0]
print_block(example_vocab_activities, "Activity (Tail removed)")
print_block(example_vocab_resources, "Resource (Tail removed)")

ground_truth_vocab =dataset.activity_vocab.index_to_vocab(example_activities[:, len(example_vocab_activities)][0])

print_block(ground_truth_vocab, "Ground Truth")


====================Activity (Tail removed)====================
| ['<SOS>', 'A_SUBMITTED_COMPLETE', 'A_PARTLYSUBMITTED_COMPLETE', 'A_PREACCEPTED_COMPLETE', 'W_Completeren aanvraag_COMPLETE', 'W_Completeren aanvraag_COMPLETE'] 

====================Resource (Tail removed)====================
| ['<SOS>', '112', '112', '112', '11180', '11201'] 

====================Ground Truth====================
| A_ACCEPTED_COMPLETE 


In [6]:
pred_model = LSTMPredNextModel.load(
    "./SavedModels/%s" % (
    "0.8175_LSTMPredNextModel_AOW_CompleteOnly_2021-07-01 20:45:16.353467" 
    )
)


| Model parameters loaded successfully from: ./SavedModels/0.8175_LSTMPredNextModel_AOW_CompleteOnly_2021-07-01 20:45:16.353467  

| Vocab loaded successfully from: ./SavedModels/0.8175_LSTMPredNextModel_AOW_CompleteOnly_2021-07-01 20:45:16.353467  

| Model loaded successfully from: ./SavedModels/0.8175_LSTMPredNextModel_AOW_CompleteOnly_2021-07-01 20:45:16.353467  


In [7]:
_ = pred_model(**pred_model.get_example_input())
pred_model.summary()

Model: "LSTMPredNextModel"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  832       
_________________________________________________________________
embedding_1 (Embedding)      multiple                  9216      
_________________________________________________________________
lstm (LSTM)                  multiple                  24832     
_________________________________________________________________
lstm_1 (LSTM)                multiple                  33024     
_________________________________________________________________
lstm_2 (LSTM)                multiple                  49408     
_________________________________________________________________
lstm_3 (LSTM)                multiple                  33024     
_________________________________________________________________
sequential (Sequential)      (1, 64)             

In [8]:
##### Get model output
example_activities_input = tf.constant(example_idx_activities)
example_resources_input = tf.constant(example_idx_resources)
example_amount_input = tf.constant(example_amount)

predicted_df = print_model_prediction_result(pred_model, example_activities_input, example_resources_input, example_amount_input)


========================================Predict result========================================
| Predicted activity with highest probability (0.69) is "W_Completeren aanvraag_COMPLETE" 



<PAD>                                      1.174409e-07
<EOS>                                      3.132308e-05
<SOS>                                      1.881654e-07
A_ACCEPTED_COMPLETE                        2.363838e-01
A_ACTIVATED_COMPLETE                       1.805911e-06
A_APPROVED_COMPLETE                        2.348171e-06
A_CANCELLED_COMPLETE                       6.621396e-02
A_DECLINED_COMPLETE                        1.187625e-02
A_FINALIZED_COMPLETE                       9.158659e-06
A_PARTLYSUBMITTED_COMPLETE                 1.821592e-07
A_PREACCEPTED_COMPLETE                     1.974105e-07
A_REGISTERED_COMPLETE                      2.050257e-06
A_SUBMITTED_COMPLETE                       1.642594e-06
O_ACCEPTED_COMPLETE                        1.552491e-05
O_CANCELLED_COMPLETE      

In [9]:
predicted_df

,<PAD>,<EOS>,<SOS>,A_ACCEPTED_COMPLETE,A_ACTIVATED_COMPLETE,A_APPROVED_COMPLETE,A_CANCELLED_COMPLETE,A_DECLINED_COMPLETE,A_FINALIZED_COMPLETE,A_PARTLYSUBMITTED_COMPLETE,...,O_DECLINED_COMPLETE,O_SELECTED_COMPLETE,O_SENT_BACK_COMPLETE,O_SENT_COMPLETE,W_Afhandelen leads_COMPLETE,W_Beoordelen fraude_COMPLETE,W_Completeren aanvraag_COMPLETE,W_Nabellen incomplete dossiers_COMPLETE,W_Nabellen offertes_COMPLETE,W_Valideren aanvraag_COMPLETE
0,1.174409e-07,0.000031,1.881654e-07,0.236384,0.000002,0.000002,0.066214,0.011876,0.000009,1.821592e-07,...,7.751900e-07,0.000033,8.179315e-08,4.313907e-07,3.938352e-07,4.016037e-07,0.685416,0.000003,5.625714e-07,0.000001


In [10]:
example_vocab_activities

['<SOS>',
 'A_SUBMITTED_COMPLETE',
 'A_PARTLYSUBMITTED_COMPLETE',
 'A_PREACCEPTED_COMPLETE',
 'W_Completeren aanvraag_COMPLETE',
 'W_Completeren aanvraag_COMPLETE']

In [11]:
example_vocab_resources

['<SOS>', '112', '112', '112', '11180', '11201']

In [12]:
example_amount

[15500.0]

In [13]:
dataset.activity_vocab.vocabs

['<PAD>',
 '<EOS>',
 '<SOS>',
 'A_ACCEPTED_COMPLETE',
 'A_ACTIVATED_COMPLETE',
 'A_APPROVED_COMPLETE',
 'A_CANCELLED_COMPLETE',
 'A_DECLINED_COMPLETE',
 'A_FINALIZED_COMPLETE',
 'A_PARTLYSUBMITTED_COMPLETE',
 'A_PREACCEPTED_COMPLETE',
 'A_REGISTERED_COMPLETE',
 'A_SUBMITTED_COMPLETE',
 'O_ACCEPTED_COMPLETE',
 'O_CANCELLED_COMPLETE',
 'O_CREATED_COMPLETE',
 'O_DECLINED_COMPLETE',
 'O_SELECTED_COMPLETE',
 'O_SENT_BACK_COMPLETE',
 'O_SENT_COMPLETE',
 'W_Afhandelen leads_COMPLETE',
 'W_Beoordelen fraude_COMPLETE',
 'W_Completeren aanvraag_COMPLETE',
 'W_Nabellen incomplete dossiers_COMPLETE',
 'W_Nabellen offertes_COMPLETE',
 'W_Valideren aanvraag_COMPLETE']

In [14]:
###### Searching for the instance.
# Define miltstones 
milestones = [
    "A_SUBMITTED_COMPLETE",
    "A_PARTLYSUBMITTED_COMPLETE",
    "A_PREACCEPTED_COMPLETE",
    "A_ACCEPTED_COMPLETE",
    "A_FINALIZED_COMPLETE",
    "O_SELECTED_COMPLETE",
    "O_CREATED_COMPLETE",
    "O_SENT_COMPLETE",
    "O_SENT_BACK_COMPLETE",
    "A_APPROVED_COMPLETE",
    "A_ACTIVATED_COMPLETE",
    "A_REGISTERED_COMPLETE"
]

In [15]:
milestone_filtered = [ a  for a in example_vocab_activities if a in milestones]
milestone_filtered

['A_SUBMITTED_COMPLETE',
 'A_PARTLYSUBMITTED_COMPLETE',
 'A_PREACCEPTED_COMPLETE']

In [16]:
desired = "A_ACCEPTED_COMPLETE"

In [17]:
query_df = training_df[[all([v in t['activity_vocab'] for v in milestone_filtered]) for t in training_df.iloc]]

only_same_amount =True

In [18]:
len(query_df)

5874

In [19]:
c = 15

In [20]:
not c is None

True

In [21]:
if only_same_amount:
    query_df = query_df[query_df['amount']==example_amount[0]]

In [22]:
# Find with desired
desired_df = query_df[[ desired in v for v in query_df['activity_vocab']]]

In [23]:
desired_df.head(3)

,activity,activity_vocab,caseid,amount,resource,resource_vocab
52,"[2, 12, 9, 10, 22, 22, 3, 8, 17, 15, 19, 22, 1...","[<SOS>, A_SUBMITTED_COMPLETE, A_PARTLYSUBMITTE...",173844,15500.0,"[70, 53, 53, 53, 50, 55, 55, 55, 55, 55, 55, 5...","[<SOS>, 112, 112, 112, 11180, 11201, 11201, 11..."
61,"[2, 12, 9, 10, 22, 22, 22, 22, 22, 3, 17, 8, 1...","[<SOS>, A_SUBMITTED_COMPLETE, A_PARTLYSUBMITTE...",173871,15500.0,"[70, 53, 53, 53, 71, 55, 18, 18, 20, 20, 20, 2...","[<SOS>, 112, 112, 112, UNKNOWN, 11201, 10889, ..."
6714,"[2, 12, 9, 10, 20, 3, 17, 8, 15, 19, 22, 24, 1...","[<SOS>, A_SUBMITTED_COMPLETE, A_PARTLYSUBMITTE...",194583,15500.0,"[70, 53, 53, 30, 30, 52, 52, 52, 52, 52, 52, 5...","[<SOS>, 112, 112, 10939, 10939, 11189, 11189, ..."


In [24]:
if (len(desired_df) <= 0):
    raise Exception("Not matches found in trainig set")

In [25]:
for idx in list(desired_df.index):
    desired_idx = desired_df.loc[idx]['activity_vocab'].index(ground_truth_vocab)

    for col in ['activity', 'activity_vocab', 'resource', 'resource_vocab']:
        desired_df.at[idx, col] = desired_df.loc[idx][col][:desired_idx]

In [26]:
all_predicted_vocabs = []
all_predicted_value = []
for idx in range(len(desired_df)):
    ex_activity = tf.constant([desired_df.iloc[idx]['activity']], dtype=tf.float32)
    ex_resource = tf.constant([desired_df.iloc[idx]['resource']], dtype=tf.float32)
    ex_amount = tf.constant([desired_df.iloc[idx]['amount']], dtype=tf.float32)
    out, _ = pred_model(ex_activity, ex_resource, ex_amount, training=False)
    out = tf.nn.softmax(out, axis=-1)
    pred_idx = tf.argmax(out[:, -1, :], axis=-1).numpy()[0]
    predicted_vocab =  dataset.activity_vocab.index_to_vocab(pred_idx)
    all_predicted_vocabs.append(predicted_vocab)
    all_predicted_value.append(out[:, -1, pred_idx].numpy()[0])

In [36]:
import pandas as pd

In [39]:
desired_df = pd.DataFrame(desired_df)

In [40]:
desired_df

,activity,activity_vocab,caseid,amount,resource,resource_vocab,Predicted
52,"[2, 12, 9, 10, 22, 22]","[<SOS>, A_SUBMITTED_COMPLETE, A_PARTLYSUBMITTE...",173844,15500.0,"[70, 53, 53, 53, 50, 55]","[<SOS>, 112, 112, 112, 11180, 11201]",W_Completeren aanvraag_COMPLETE
61,"[2, 12, 9, 10, 22, 22, 22, 22, 22]","[<SOS>, A_SUBMITTED_COMPLETE, A_PARTLYSUBMITTE...",173871,15500.0,"[70, 53, 53, 53, 71, 55, 18, 18, 20]","[<SOS>, 112, 112, 112, UNKNOWN, 11201, 10889, ...",W_Completeren aanvraag_COMPLETE
6714,"[2, 12, 9, 10, 20]","[<SOS>, A_SUBMITTED_COMPLETE, A_PARTLYSUBMITTE...",194583,15500.0,"[70, 53, 53, 30, 30]","[<SOS>, 112, 112, 10939, 10939]",W_Completeren aanvraag_COMPLETE
1031,"[2, 12, 9, 10, 20]","[<SOS>, A_SUBMITTED_COMPLETE, A_PARTLYSUBMITTE...",176927,15500.0,"[70, 53, 53, 38, 38]","[<SOS>, 112, 112, 11009, 11009]",W_Completeren aanvraag_COMPLETE
7405,"[2, 12, 9, 10]","[<SOS>, A_SUBMITTED_COMPLETE, A_PARTLYSUBMITTE...",196777,15500.0,"[70, 53, 53, 53]","[<SOS>, 112, 112, 112]",W_Completeren aanvraag_COMPLETE
583,"[2, 12, 9, 10, 22]","[<SOS>, A_SUBMITTED_COMPLETE, A_PARTLYSUBMITTE...",175528,15500.0,"[70, 53, 53, 53, 22]","[<SOS>, 112, 112, 112, 10912]",W_Completeren aanvraag_COMPLETE
7554,"[2, 12, 9, 10, 20]","[<SOS>, A_SUBMITTED_COMPLETE, A_PARTLYSUBMITTE...",197234,15500.0,"[70, 53, 53, 25, 25]","[<SOS>, 112, 112, 10929, 10929]",W_Completeren aanvraag_COMPLETE
9470,"[2, 12, 9, 10, 20]","[<SOS>, A_SUBMITTED_COMPLETE, A_PARTLYSUBMITTE...",203206,15500.0,"[70, 53, 53, 21, 21]","[<SOS>, 112, 112, 10910, 10910]",W_Completeren aanvraag_COMPLETE
7786,"[2, 12, 9, 10]","[<SOS>, A_SUBMITTED_COMPLETE, A_PARTLYSUBMITTE...",197948,15500.0,"[70, 53, 53, 53]","[<SOS>, 112, 112, 112]",W_Completeren aanvraag_COMPLETE


In [38]:
desired_df.loc[:, "Predicted"] = all_predicted_vocabs

In [28]:
desired_df

,activity,activity_vocab,caseid,amount,resource,resource_vocab,Predicted
52,"[2, 12, 9, 10, 22, 22]","[<SOS>, A_SUBMITTED_COMPLETE, A_PARTLYSUBMITTE...",173844,15500.0,"[70, 53, 53, 53, 50, 55]","[<SOS>, 112, 112, 112, 11180, 11201]",W_Completeren aanvraag_COMPLETE
61,"[2, 12, 9, 10, 22, 22, 22, 22, 22]","[<SOS>, A_SUBMITTED_COMPLETE, A_PARTLYSUBMITTE...",173871,15500.0,"[70, 53, 53, 53, 71, 55, 18, 18, 20]","[<SOS>, 112, 112, 112, UNKNOWN, 11201, 10889, ...",W_Completeren aanvraag_COMPLETE
6714,"[2, 12, 9, 10, 20]","[<SOS>, A_SUBMITTED_COMPLETE, A_PARTLYSUBMITTE...",194583,15500.0,"[70, 53, 53, 30, 30]","[<SOS>, 112, 112, 10939, 10939]",W_Completeren aanvraag_COMPLETE
1031,"[2, 12, 9, 10, 20]","[<SOS>, A_SUBMITTED_COMPLETE, A_PARTLYSUBMITTE...",176927,15500.0,"[70, 53, 53, 38, 38]","[<SOS>, 112, 112, 11009, 11009]",W_Completeren aanvraag_COMPLETE
7405,"[2, 12, 9, 10]","[<SOS>, A_SUBMITTED_COMPLETE, A_PARTLYSUBMITTE...",196777,15500.0,"[70, 53, 53, 53]","[<SOS>, 112, 112, 112]",W_Completeren aanvraag_COMPLETE
583,"[2, 12, 9, 10, 22]","[<SOS>, A_SUBMITTED_COMPLETE, A_PARTLYSUBMITTE...",175528,15500.0,"[70, 53, 53, 53, 22]","[<SOS>, 112, 112, 112, 10912]",W_Completeren aanvraag_COMPLETE
7554,"[2, 12, 9, 10, 20]","[<SOS>, A_SUBMITTED_COMPLETE, A_PARTLYSUBMITTE...",197234,15500.0,"[70, 53, 53, 25, 25]","[<SOS>, 112, 112, 10929, 10929]",W_Completeren aanvraag_COMPLETE
9470,"[2, 12, 9, 10, 20]","[<SOS>, A_SUBMITTED_COMPLETE, A_PARTLYSUBMITTE...",203206,15500.0,"[70, 53, 53, 21, 21]","[<SOS>, 112, 112, 10910, 10910]",W_Completeren aanvraag_COMPLETE
7786,"[2, 12, 9, 10]","[<SOS>, A_SUBMITTED_COMPLETE, A_PARTLYSUBMITTE...",197948,15500.0,"[70, 53, 53, 53]","[<SOS>, 112, 112, 112]",W_Completeren aanvraag_COMPLETE


In [29]:
from cf_search import CfSearcher

In [30]:
searcher = CfSearcher(training_df, pred_model)

In [32]:
searcher.search(example_vocab_activities, "A_ACCEPTED_COMPLETE", amount=example_amount[0])

/Users/jrhs/EventLogDiCE/cf_search/CfSearcher.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  desired_df['Predicted_vocab'] = all_predicted_vocabs
/Users/jrhs/EventLogDiCE/cf_search/CfSearcher.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  desired_df['Predicted_value'] = all_predicted_value


,activity,activity_vocab,caseid,amount,resource,resource_vocab,Predicted_vocab,Predicted_value
52,"[2, 12, 9, 10, 22, 22]","[<SOS>, A_SUBMITTED_COMPLETE, A_PARTLYSUBMITTE...",173844,15500.0,"[70, 53, 53, 53, 50, 55]","[<SOS>, 112, 112, 112, 11180, 11201]",W_Completeren aanvraag_COMPLETE,0.685416
61,"[2, 12, 9, 10, 22, 22, 22, 22, 22]","[<SOS>, A_SUBMITTED_COMPLETE, A_PARTLYSUBMITTE...",173871,15500.0,"[70, 53, 53, 53, 71, 55, 18, 18, 20]","[<SOS>, 112, 112, 112, UNKNOWN, 11201, 10889, ...",W_Completeren aanvraag_COMPLETE,0.878228
6714,"[2, 12, 9, 10, 20]","[<SOS>, A_SUBMITTED_COMPLETE, A_PARTLYSUBMITTE...",194583,15500.0,"[70, 53, 53, 30, 30]","[<SOS>, 112, 112, 10939, 10939]",W_Completeren aanvraag_COMPLETE,0.528128
1031,"[2, 12, 9, 10, 20]","[<SOS>, A_SUBMITTED_COMPLETE, A_PARTLYSUBMITTE...",176927,15500.0,"[70, 53, 53, 38, 38]","[<SOS>, 112, 112, 11009, 11009]",W_Completeren aanvraag_COMPLETE,0.671937
7405,"[2, 12, 9, 10]","[<SOS>, A_SUBMITTED_COMPLETE, A_PARTLYSUBMITTE...",196777,15500.0,"[70, 53, 53, 53]","[<SOS>, 112, 112, 112]",W_Completeren aanvraag_COMPLETE,0.555065
583,"[2, 12, 9, 10, 22]","[<SOS>, A_SUBMITTED_COMPLETE, A_PARTLYSUBMITTE...",175528,15500.0,"[70, 53, 53, 53, 22]","[<SOS>, 112, 112, 112, 10912]",W_Completeren aanvraag_COMPLETE,0.652978
7554,"[2, 12, 9, 10, 20]","[<SOS>, A_SUBMITTED_COMPLETE, A_PARTLYSUBMITTE...",197234,15500.0,"[70, 53, 53, 25, 25]","[<SOS>, 112, 112, 10929, 10929]",W_Completeren aanvraag_COMPLETE,0.608703
9470,"[2, 12, 9, 10, 20]","[<SOS>, A_SUBMITTED_COMPLETE, A_PARTLYSUBMITTE...",203206,15500.0,"[70, 53, 53, 21, 21]","[<SOS>, 112, 112, 10910, 10910]",W_Completeren aanvraag_COMPLETE,0.634169
7786,"[2, 12, 9, 10]","[<SOS>, A_SUBMITTED_COMPLETE, A_PARTLYSUBMITTE...",197948,15500.0,"[70, 53, 53, 53]","[<SOS>, 112, 112, 112]",W_Completeren aanvraag_COMPLETE,0.555065
